# TD2 part 2: Named entity recognition

Dans ce TD, nous allons prendre un datasets où les noms de personnes sont taggés.<br>
Nous allons transformer ces données en tenseurs X, y et attention_mask.<br>
Nous allons créer un modèle RNN pour prédire si un mot est un nom de personne.<br>
Nous allons ensuite créer la loop avec l'optimizer pour apprendre le modèle.<br>
Du modèle appris (prédisant sur les tokens), nous allons postprocess les prédictions pour avoir les prédictions sur les noms.

Un fois que la loop est créée et que le modèle apprend, nous allons changer la structure du modèle:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.

Lorsqu'on a un bon modèle de prédiction pour les noms de personnes, nous allons l'appliquer à notre projet fil rouge.
Utilisez-le tel que. Quelle accuracy ?
Ré-entrainez la (les) dernière(s) couche(s) du modèle sur notre jeu de données. A-t-il gagné en accuracy ?

In [2]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.9 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.6 MB 38.1 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 55.4 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 841 kB/s  eta 0:00:01
     |████████████████████████████████| 311 kB 26.7 MB/s eta 0:00:01
     |████████████████████████████████| 439 kB 30.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 448 kB/s  eta 0:00:01
     |████████████████████████████████| 122 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 59.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import
import matplotlib.pyplot as plt
import numpy as np
import torch
import transformers

/Users/stockly/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/stockly/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

Télécharger le dataset MultiNERD FR [ici](https://github.com/Babelscape/multinerd)<br>
Mettez les données dans le dossier data/raw du projet.


In [2]:
def extract_multinerd_person_words(filename="./data/raw/train_fr.tsv"):
    with open(filename) as f:
        tagged_words = [line.strip().split("\t") for line in f]
        
        # Joining words until we meet a dot
        # Word's label is 1 if 'PER' is in its tag
        sentences = []
        sentence_labels = []
        this_word = []
        this_labels = []
        for tagged_word in tagged_words:
            if len(tagged_word) < 3:
                # not a tagged word
                continue
            word = tagged_word[1]
            tag = tagged_word[2]
        
            if word == '.':
                sentences.append(" ".join(this_word))
                sentence_labels.append(this_labels)
            
                this_word = []
                this_labels = []
            else:
                this_word.append(word)
                this_labels.append(1 * tag.endswith("PER"))

    return sentences, sentence_labels


In [3]:
sentences, labels = extract_multinerd_person_words("./data/raw/train_fr.tsv")

## Tokenizer

En utilisant le tokenizer d'HuggingFace "camembert-base":
- Transformer les phrases en tokens
- Obtenez des vecteur y qui ont le même nombre d'entrées qu'il y a de tokens dans la phrase
- Ayez un tenseur "attention_mask" pour savoir sur quels tokens on cherche à predire le label
- Transformez les tokens en token_ids (avec le tokenizer)
Avec tout cela, vous pouvez former vos tenseurs X, Y et attention_mask

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [5]:
def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    words = sentence.split()
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for word, label in zip(words, labels):
        this_tokens = tokenizer.tokenize(word)
        tokens += this_tokens
        
        this_labels = [0] * len(this_tokens)
        this_labels[0] = label        
        tokens_label += this_labels
        
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    return tokens, tokens_label, attention_mask

In [6]:
tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentences[0], labels[0])

In [7]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    # Tokenize the sentence
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sentence)))
    
    # Initialize labels and attention mask lists
    tokens_label = []
    attention_mask = []
    
    # Process each word and its corresponding label
    for word, label in zip(tokens, labels):
        # Tokenize the word
        this_tokens = tokenizer.tokenize(word)
        tokens_label += [label] * len(this_tokens)
        
        # Create attention mask
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    # Encode the tokens to obtain token IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    
    return token_ids, tokens_label, attention_mask

# Example usage
sentence = "Bonjour le monde"
labels = [1, 0, 1, 0]
token_ids, tokens_label, attention_mask = build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels)

# Convert lists to tensors
token_ids = torch.tensor(token_ids)
tokens_label = torch.tensor(tokens_label)
attention_mask = torch.tensor(attention_mask)

print("Token IDs:", token_ids)
print("Tokens Label:", tokens_label)
print("Attention Mask:", attention_mask)


2023-12-05 16:14:28.930325: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Token IDs: tensor([   5, 1285,   16,  164,    6])
Tokens Label: tensor([1, 0, 1, 0])
Attention Mask: tensor([1, 1, 1, 1])


In [8]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=20, tagset_size=2):
        super(RNNModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)

        self.relu = nn.ReLU()

        self.linear = nn.Linear(hidden_dim, tagset_size)

        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids):
        embedded = self.embedding(input_ids)

        rnn_out, _ = self.rnn(embedded)

        rnn_out = rnn_out[:, -1, :]

        relu_out = self.relu(rnn_out)

        linear_out = self.linear(relu_out)

        output = self.softmax(linear_out)

        return output

vocab_size = 52000  
model = RNNModel(vocab_size)

input_ids = torch.tensor([[1, 2, 3, 4, 5]])  

output = model(input_ids)
print(output)


tensor([[-0.3432, -1.2361]], grad_fn=<LogSoftmaxBackward0>)


## Model

Contruisez un modèle RNN comme dans la partie 1. Pour l'instant, il prendra comme arguments:
- Vocab size: le nombre de différents tokens du tokenizer (52 000 pour camembert-base)
- Embedding dim: la dimension de l'embedding des tokens (par défaut 50)
- hidden_dim: la dimension de l'état récurrent de votre RNN (par défaut 20)
- tagset_size: la nombre de classes possibles pour les prédictions (ici 2)

Dans le forward, votre modèle enchaînera les couches suivantes:
- Un embedding
- Un RNN
- Un ReLU
- Une couche linéaire
- Un softmax pour que la somme des prédictions pour une entrée soit égale à 1 (la prédiction pour un élément et sa probabilité d'être dans chaque classe)

In [10]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=20, tagset_size=2):
        super(RNNModel, self).__init__()
        
        # Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        
        # RNN Layer
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        
        # ReLU Activation
        self.relu = nn.ReLU()
        
        # Linear Layer
        self.linear = nn.Linear(hidden_dim, tagset_size)
        
        # Softmax Activation
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_ids):
        # Embedding
        embedded = self.embedding(input_ids)
        
        # RNN
        rnn_out, _ = self.rnn(embedded)
        
        # Select the output of the last time step
        rnn_out = rnn_out[:, -1, :]
        
        # ReLU activation
        relu_out = self.relu(rnn_out)
        
        # Linear layer
        linear_out = self.linear(relu_out)
        
        # Softmax activation
        output = self.softmax(linear_out)
        
        return output


## Optimizer

Je fournis ici une fonction prenant un modèle, des tenseurs X, y, attention_mask.
Pour chaque batch:
- La loop utilise le modèle pour prédire sur x_batch
- Avec attention_mask, elle identifie sur quels tokens les prédictions compte
- Elle regarde la cross entropy entre y\[attention_ids\] et yhat\[attention_ids\]
- Elle output un dictionnaire avec le model et la loss au fur et à mesure des itérations

Entraînez le modèle avec vos données. <br>
Plottez la loss history.<br>
Itérez sur le modèle pour l'améliorer:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.



In [11]:
def train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128):
    loss_function = torch.nn.CrossEntropyLoss()
    loss_history = []

    dataset = torch.utils.data.TensorDataset(X, y, attention_masks)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for i, (x_batch, y_batch, mask) in enumerate(loader):
            optimizer.zero_grad()
        
            ids = mask.reshape(-1)
            yhat = model(x_batch).reshape((-1, 2))[ids]
            this_y = y_batch.reshape(-1)[ids]
            
            loss = loss_function(yhat, this_y)
            loss.backward()
            
            loss_history.append(loss.clone().detach())
        
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Got loss at {epoch}", np.mean(loss_history[-10:]))
    
    return {"model": model, "loss_history": loss_history}

In [12]:
import torch.optim as optim

def train_model(model, X, y, attention_mask, learning_rate=0.001, num_epochs=10):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    loss_history = []

    for epoch in range(num_epochs):
        epoch_loss = 0.0

        for x_batch, y_batch, mask_batch in zip(X, y, attention_mask):
            model.train()

            optimizer.zero_grad()

            outputs = model(x_batch)

            attention_ids = torch.nonzero(mask_batch).view(-1)

            loss = criterion(outputs[attention_ids], y_batch[attention_ids])

            loss.backward()

            optimizer.step()

            epoch_loss += loss.item()

        epoch_loss /= len(X)

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
        loss_history.append(epoch_loss)

    print('Entraînement terminé!')
    return model, loss_history


## Postprocessing

Créer une fonction prenant les prédictions du modèle (au niveau token) et sort les prédictions au niveau mot.<br>
Par exemple, admettons que, pour un mot, la prédiction du 1er token est la seule qu'on considère.<br>
si la phrase est "Bonjour John", avec les tokens \["bon", "jour", "Jo", "hn"\] avec les predictions \[0.12, 0.65, 0.88, 0.45\]<br>
Je veux récupérer les prédictions "bonjour": 0.12, "John": 0.88

In [13]:
def postprocess_predictions(token_predictions, tokens, word_indices):
    word_predictions = {}

    for word, start_idx, end_idx in word_indices:
        word_token_preds = token_predictions[start_idx:end_idx + 1]
        
        word_prediction = word_token_preds[0]

        word_predictions[word] = word_prediction

    return word_predictions

token_predictions = [0.12, 0.65, 0.88, 0.45]
tokens = ["bon", "jour", "Jo", "hn"]
word_indices = [("bonjour", 0, 1), ("John", 2, 3)]

result = postprocess_predictions(token_predictions, tokens, word_indices)
print(result)


{'bonjour': 0.12, 'John': 0.88}
